In [118]:
from dotenv import load_dotenv
import os
import pandas as pd

# Load environment variables from .env file
load_dotenv()
from tqdm.auto import tqdm

# Access environment variables
openai_key = os.getenv('OPENAI_API_KEY')

In [119]:
from elasticsearch import Elasticsearch

In [120]:
from openai import OpenAI

client = OpenAI()

In [121]:
# Initialize Elasticsearch client
es = Elasticsearch("http://localhost:9200")
index_name="products"

In [122]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

/home/joyce/.local/share/virtualenvs/Ecommerce_Product_Search_Assistant-nqyLfLy5/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [124]:
# Text search
def full_text_search(query):
    response = es.search(
        index=index_name,
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "description"]
                }
            }
        }
    )
    for result in response['hits']['hits']:
        return [result]

query = "sports outfit for women."
result = full_text_search(query)
print(result)

[{'_index': 'products', '_id': 'E4h4n5EB2ZIsYP2mcUTk', '_score': 9.604138, '_source': {'title': 'Sports Shoes For Men\xa0\xa0(Multicolor)', 'description': 'Vector X Is One Of The Established Sports Brands Who Provide Stylish And Innovative Sports Gear To Most Of The Aspiring Sport Players As Well As Others. As A Brand They Have Produced Thousands Of Spectacular Gear Which Have Been Used By Numerous Renowned', 'category': 'Footwear', 'sub_category': "Men's Footwear", 'brand': 'Vector', 'doc_id': 3635, 'product_details': {'Color': 'Multicolor'}, 'average_rating': '3.7', 'url': 'https://www.flipkart.com/vector-x-sports-shoes-men/p/itmf3xz7eesu3eqn?pid=SHOE7TZFDJF8WJTC&lid=LSTSHOE7TZFDJF8WJTCXLMNQS&marketplace=FLIPKART&srno=b_2_74&otracker=nmenu_sub_Men_0_Footwear&fm=organic&iid=ed342989-9f17-43c2-b620-87aa4485e449.SHOE7TZFDJF8WJTC.SEARCH&ssid=3u6pawgow00000001612688548473', 'out_of_stock': False, 'title_and_description_vector': [-0.018957842141389847, -0.05382869765162468, -0.007869360037

In [125]:
# Text search
def full_text_search_with_title_boosting(query):
    response = es.search(
        index=index_name,
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["title^3", "description"]
                }
            }
        }
    )
    for result in response['hits']['hits']:
        return [result]

query = "sports outfit for women."
result = full_text_search(query)
print(result)

[{'_index': 'products', '_id': 'E4h4n5EB2ZIsYP2mcUTk', '_score': 9.604138, '_source': {'title': 'Sports Shoes For Men\xa0\xa0(Multicolor)', 'description': 'Vector X Is One Of The Established Sports Brands Who Provide Stylish And Innovative Sports Gear To Most Of The Aspiring Sport Players As Well As Others. As A Brand They Have Produced Thousands Of Spectacular Gear Which Have Been Used By Numerous Renowned', 'category': 'Footwear', 'sub_category': "Men's Footwear", 'brand': 'Vector', 'doc_id': 3635, 'product_details': {'Color': 'Multicolor'}, 'average_rating': '3.7', 'url': 'https://www.flipkart.com/vector-x-sports-shoes-men/p/itmf3xz7eesu3eqn?pid=SHOE7TZFDJF8WJTC&lid=LSTSHOE7TZFDJF8WJTCXLMNQS&marketplace=FLIPKART&srno=b_2_74&otracker=nmenu_sub_Men_0_Footwear&fm=organic&iid=ed342989-9f17-43c2-b620-87aa4485e449.SHOE7TZFDJF8WJTC.SEARCH&ssid=3u6pawgow00000001612688548473', 'out_of_stock': False, 'title_and_description_vector': [-0.018957842141389847, -0.05382869765162468, -0.007869360037

In [126]:
# Text search
def full_text_search_with_description_boosting(query):
    response = es.search(
        index=index_name,
        body={
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": ["title", "description^2"]
                }
            }
        }
    )
    for result in response['hits']['hits']:
        return [result]

query = "sports outfit for women."
result = full_text_search(query)
print(result)

[{'_index': 'products', '_id': 'E4h4n5EB2ZIsYP2mcUTk', '_score': 9.604138, '_source': {'title': 'Sports Shoes For Men\xa0\xa0(Multicolor)', 'description': 'Vector X Is One Of The Established Sports Brands Who Provide Stylish And Innovative Sports Gear To Most Of The Aspiring Sport Players As Well As Others. As A Brand They Have Produced Thousands Of Spectacular Gear Which Have Been Used By Numerous Renowned', 'category': 'Footwear', 'sub_category': "Men's Footwear", 'brand': 'Vector', 'doc_id': 3635, 'product_details': {'Color': 'Multicolor'}, 'average_rating': '3.7', 'url': 'https://www.flipkart.com/vector-x-sports-shoes-men/p/itmf3xz7eesu3eqn?pid=SHOE7TZFDJF8WJTC&lid=LSTSHOE7TZFDJF8WJTCXLMNQS&marketplace=FLIPKART&srno=b_2_74&otracker=nmenu_sub_Men_0_Footwear&fm=organic&iid=ed342989-9f17-43c2-b620-87aa4485e449.SHOE7TZFDJF8WJTC.SEARCH&ssid=3u6pawgow00000001612688548473', 'out_of_stock': False, 'title_and_description_vector': [-0.018957842141389847, -0.05382869765162468, -0.007869360037

In [127]:
# semantic search
def semantic_search(query):
    query_vector = model.encode(query)
    
    query = {
        "field": "title_and_description_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 1000, 
    }
    
    res = es.search(index=index_name, knn=query, source=["title", "description", "category", "sub_category", "brand", "doc_id"])
    return res["hits"]["hits"]
    
result = semantic_search(query)
print(result)

[{'_index': 'products', '_id': 'XYh5n5EB2ZIsYP2mAFi7', '_score': 0.790504, '_source': {'sub_category': 'Clothing Accessories', 'description': 'Self Design Unisex Baseball NY Logo Stylish Cotton Cap', 'title': 'Solid New BLACK Design Half Net for Boy girls Cap', 'category': 'Clothing and Accessories', 'brand': 'R', 'doc_id': 8829}}, {'_index': 'products', '_id': 'Zoh6n5EB2ZIsYP2m5Jkp', '_score': 0.79036194, '_source': {'sub_category': 'Winter Wear', 'description': 'Woven TFS label on the flap;Archive No. 2 Logo rubber print on the front;Archive No. 1 Logo rubber print on the back;Stand-up-collar;Flap over zipper;AOP pannels on front and back;Single-welt side pockets for storage solutions;Elasticated cuffs and hem;Relaxed fit;Made with bluesign certified fabrics;1990, meet 2020. Tailored for Sports is rooted in our PUMA Archive collection from Fall/Winter 1990, which was designed to bring together sport, function, and fashion for the first time. Today, the new collection hails a more col

In [128]:
# hybrid search
def hybrid_search(query, brand):
    query_vector = model.encode(query)
    knn_query = {
        "field": "title_and_description_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 1000
    }
    
    response = es.search(
        index=index_name,
        query={
            "match": {"brand": brand},
        },
        knn=knn_query,
        size=5
    )
    return response["hits"]["hits"]

category="Clothing and Accessories"
query = "sports outfit for women."

result = hybrid_search(query, category)
print(result)

[{'_index': 'products', '_id': 'XYh5n5EB2ZIsYP2mAFi7', '_score': 0.790504, '_source': {'title': 'Solid New BLACK Design Half Net for Boy girls Cap', 'description': 'Self Design Unisex Baseball NY Logo Stylish Cotton Cap', 'category': 'Clothing and Accessories', 'sub_category': 'Clothing Accessories', 'brand': 'R', 'doc_id': 8829, 'product_details': {'Fabric': 'Polycotton'}, 'average_rating': '3.6', 'url': 'https://www.flipkart.com/roy-solid-new-black-design-half-net-boy-girls-cap/p/itmf8rmqffvfzywe?pid=CAPF8QM3PFA2ZCQ8&lid=LSTCAPF8QM3PFA2ZCQ8HW1EOG&marketplace=FLIPKART&srno=b_1_1&otracker=browse&fm=organic&iid=c608389d-d81b-484d-8166-26019458cb83.CAPF8QM3PFA2ZCQ8.SEARCH&ssid=wla6dcz2c00000001612413004577', 'out_of_stock': False, 'title_and_description_vector': [0.006994473747909069, 0.01994452439248562, -0.022943299263715744, 0.015501521527767181, 0.015599562786519527, -0.0369354784488678, 0.010533059947192669, 0.011631401255726814, -0.0403200127184391, -0.007203129585832357, 0.0683738

In [129]:
# hybrid search
def hybrid_search_with_brand_and_category(query):
    query_vector = model.encode(query)
    knn_query = {
        "field": "title_and_description_vector",
        "query_vector": query_vector,
        "k": 5,
        "num_candidates": 1000
    }
    
    response = es.search(
        index=index_name,
        query={
            "multi_match": {
                    "query": query,
                    "fields": ["brand", "category"]
                }
        },
        knn=knn_query,
        size=5
    )
    return response["hits"]["hits"]

title="York"
result = hybrid_search(query, title)
print(result)

[{'_index': 'products', '_id': '4Ih3n5EB2ZIsYP2mxDVp', '_score': 6.701889, '_source': {'title': 'Solid Men Multicolor Track Pants', 'description': 'Yorker trackpants made from 100% rich combed cotton giving it a rich look.Designed for Comfort,Skin friendly fabric,itch-free waistband & great for all year round use Proudly made in India', 'category': 'Clothing and Accessories', 'sub_category': 'Bottomwear', 'brand': 'York', 'doc_id': 0, 'product_details': {'Style Code': '1005COMBO2'}, 'average_rating': '3.9', 'url': 'https://www.flipkart.com/yorker-solid-men-multicolor-track-pants/p/itmd2c76aadce459?pid=TKPFCZ9EA7H5FYZH&lid=LSTTKPFCZ9EA7H5FYZHVYXWP0&marketplace=FLIPKART&srno=b_1_1&otracker=browse&fm=organic&iid=177a46eb-d053-4732-b3de-fcad6ff59cbd.TKPFCZ9EA7H5FYZH.SEARCH&ssid=utkd4t3gb40000001612415717799', 'out_of_stock': False, 'title_and_description_vector': [-0.04162813723087311, 0.003835424780845642, -0.0027257739566266537, 0.023723645135760307, -0.03532840684056282, -0.015687942504

In [130]:
prompt_template = """
You're the product search assistant for an ecommerce website. Answer the QUESTION based on the CONTEXT from our products database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
title: {title}
description: {description}
category: {category}
sub_category: {sub_category}
brand: {brand}
average rating: {average_rating}
url: {url}
out_of_stock: {out_of_stock}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc['_source']) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [131]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [132]:
def rag(query, model='gpt-4o-mini'):
    search_results = full_text_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer

In [133]:
question = "What is a good pants available?"
answer = rag(question)
print(answer)

A good pair of pants available is the Slim Men White Jeans from Reliable Cart. They are crafted in soft denim with a slim fit, featuring stylish side strips and logos. The jeans are made from real denim for added comfort and are suitable for regular wear. It has an average rating of 4.1. You can view them [here](https://www.flipkart.com/reliable-cart-slim-men-white-jeans/p/itm646c6cef5aa39?pid=JEAFVMMSRYBGDQQ2&lid=LSTJEAFVMMSRYBGDQQ2GBRTLH&marketplace=FLIPKART&srno=b_1_3&otracker=browse&fm=organic&iid=69c3e1f6-c4a6-4a5e-a152-0dcf6119f27c.JEAFVMMSRYBGDQQ2.SEARCH&ssid=lzwkz0lag00000001612412560887).


## Retrieval Evaluation

In [134]:

df_question = pd.read_csv('../data/ground-truth-retrieval.csv')


In [135]:
df_question.head()


,doc_id,questions,category,brand
0,0,What are the material and comfort features of ...,Clothing and Accessories,York
1,0,Is the Solid Men Multicolor Track Pants by Yor...,Clothing and Accessories,York
2,1,What materials are used to make the Solid Men ...,Clothing and Accessories,York
3,1,Is the waistband of the Yorker track pants des...,Clothing and Accessories,York
4,2,What material is used in the York Solid Men Mu...,Clothing and Accessories,York


In [136]:
ground_truth = df_question.to_dict(orient='records')


In [137]:
ground_truth[0]


{'doc_id': 0,
 'questions': 'What are the material and comfort features of the Solid Men Multicolor Track Pants by York?',
 'category': 'Clothing and Accessories',
 'brand': 'York'}

In [138]:
df_question.tail(10)

,doc_id,questions,category,brand
30,15,What is the average rating of the Solid Men Mu...,Clothing and Accessories,York
31,15,Is the Solid Men Multicolor Track Pants curren...,Clothing and Accessories,York
32,16,What is the average rating of the Self Design ...,Clothing and Accessories,York
33,16,Is the Self Design Men Multicolor Track Pants ...,Clothing and Accessories,York
34,17,What is the average rating of the Self Design ...,Clothing and Accessories,York
35,17,Is the Self Design Men Multicolor Track Pants ...,Clothing and Accessories,York
36,18,What is the average rating of the York Self De...,Clothing and Accessories,York
37,18,Is the York Self Design Men Multicolor Track P...,Clothing and Accessories,York
38,19,What materials are used in the York Solid Men ...,Clothing and Accessories,York
39,19,Is the York Solid Men Multicolor Track Pants a...,Clothing and Accessories,York


In [139]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [140]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['doc_id']
        results = search_function(q)
        # print(results)
        relevance = [d['_source']['doc_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [141]:
ground_truth

[{'doc_id': 0,
  'questions': 'What are the material and comfort features of the Solid Men Multicolor Track Pants by York?',
  'category': 'Clothing and Accessories',
  'brand': 'York'},
 {'doc_id': 0,
  'questions': 'Is the Solid Men Multicolor Track Pants by York suitable for year-round use?',
  'category': 'Clothing and Accessories',
  'brand': 'York'},
 {'doc_id': 1,
  'questions': 'What materials are used to make the Solid Men Blue Track Pants by York?',
  'category': 'Clothing and Accessories',
  'brand': 'York'},
 {'doc_id': 1,
  'questions': 'Is the waistband of the Yorker track pants designed to be comfortable for all-day wear?',
  'category': 'Clothing and Accessories',
  'brand': 'York'},
 {'doc_id': 2,
  'questions': 'What material is used in the York Solid Men Multicolor Track Pants and how does it enhance comfort?',
  'category': 'Clothing and Accessories',
  'brand': 'York'},
 {'doc_id': 2,
  'questions': 'Is the waistband of the York trackpants itch-free, and are they s

In [142]:
## Evaluate full-text search
evaluate(ground_truth, lambda q: full_text_search(q['questions']))


  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.025, 'mrr': 0.025}

In [143]:
## Evaluate semantic search
evaluate(ground_truth, lambda q: semantic_search(q['questions']))

  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.425, 'mrr': 0.2554166666666666}

In [144]:
## Evaluate hybrid search
evaluate(ground_truth, lambda q: hybrid_search(q['questions'], brand=q['brand']))

  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.55, 'mrr': 0.3350000000000001}

In [145]:
## Evaluate hybrid search with brand and category
evaluate(ground_truth, lambda q: hybrid_search_with_brand_and_category(q['questions']))

  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.425, 'mrr': 0.2554166666666666}

In [146]:
## Evaluate full-text search with boosting on title field
evaluate(ground_truth, lambda q: full_text_search_with_title_boosting(q['questions']))

  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.275, 'mrr': 0.275}

In [147]:
## Evaluate full-text search with boosting on description field
evaluate(ground_truth, lambda q: full_text_search_with_description_boosting(q['questions']))

  0%|          | 0/40 [00:00<?, ?it/s]

{'hit_rate': 0.0, 'mrr': 0.0}

In [108]:
# RAG Evaluation
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT""
}}
""".strip()

In [71]:
len(ground_truth)


40

In [75]:
ground_truth[0]

{'doc_id': 0,
 'questions': 'What are the material and comfort features of the Solid Men Multicolor Track Pants by York?',
 'category': 'Clothing and Accessories',
 'brand': 'York'}

In [ ]:
def rag(query, model='gpt-4o-mini'):
    search_results = full_text_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer

In [81]:
question = ground_truth[0]['questions']
answer_llm = rag(record)
answer_llm

'The Solid Men Multicolor Track Pants by York are made from 100% rich combed cotton, providing a rich look. For comfort, they feature a skin-friendly fabric, an itch-free waistband, and are designed for great comfort suitable for all year round use.'

In [82]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: What are the material and comfort features of the Solid Men Multicolor Track Pants by York?
Generated Answer: The Solid Men Multicolor Track Pants by York are made from 100% rich combed cotton, providing a rich look. For comfort, they feature a skin-friendly fabric, an itch-free waistband, and are designed for great comfort suitable for all year round use.

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}


In [83]:
import json

In [99]:
rag_evals = pd.DataFrame(df_question)
rag_evals

,doc_id,questions,category,brand
0,0,What are the material and comfort features of ...,Clothing and Accessories,York
1,0,Is the Solid Men Multicolor Track Pants by Yor...,Clothing and Accessories,York
2,1,What materials are used to make the Solid Men ...,Clothing and Accessories,York
3,1,Is the waistband of the Yorker track pants des...,Clothing and Accessories,York
4,2,What material is used in the York Solid Men Mu...,Clothing and Accessories,York
5,2,Is the waistband of the York trackpants itch-f...,Clothing and Accessories,York
6,3,What materials are used in the Yorker Solid Me...,Clothing and Accessories,York
7,3,Are the Yorker track pants comfortable for all...,Clothing and Accessories,York
8,4,What materials are used in the York Solid Men ...,Clothing and Accessories,York
9,4,Can you tell me about the sizing options avail...,Clothing and Accessories,York


In [104]:
question = 'What are the material and comfort features of the Solid Men Multicolor Track Pants by York?'
ans_llm = rag(question) 
print(ans_llm)

The Solid Men Multicolor Track Pants by York are made from 100% rich combed cotton, providing a rich look. They are designed for comfort, featuring skin-friendly fabric and an itch-free waistband, making them suitable for all-year-round use.


In [109]:
prompt = prompt2_template.format(
        question=question,
        answer_llm=ans_llm
    )

evaluation = llm(prompt)
    # evaluation = json.loads(evaluation)


In [110]:
print(evaluation)

{
  "Relevance": "RELEVANT"
}


In [113]:
def evaluate_rag(row):
    question = row['questions']
    answer_llm = rag(question) 
    prompt = prompt2_template.format(
        question=question,
        answer_llm=ans_llm
    )

    evaluation = llm(prompt)
    return evaluation

In [114]:
rag_evals['eval'] = rag_evals.apply(lambda x: evaluate_rag(x), axis=1)
rag_evals

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-3eWPMjDXUV5cFTFupKPdQTGd on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [115]:
rag_evals['eval'].value_counts(normalize=True)

eval
What is the average rating of the York Self Design Men Multicolor Track Pants?                                    0.075
Is the York Self Design Men Multicolor Track Pants currently in stock?                                            0.075
What is the average rating of the Self Design Men Multicolor Track Pants by York?                                 0.050
Is the Self Design Men Multicolor Track Pants currently in stock?                                                 0.050
What is the average rating of the Solid Men Multicolor Track Pants by York?                                       0.050
What are the material and comfort features of the Solid Men Multicolor Track Pants by York?                       0.025
Is the Solid Men Multicolor Track Pants by York suitable for year-round use?                                      0.025
Is the waistband of the Yorker track pants designed to be comfortable for all-day wear?                           0.025
What materials are used to make the

In [117]:
rag_evals

,doc_id,questions,category,brand,eval
0,0,What are the material and comfort features of ...,Clothing and Accessories,York,What are the material and comfort features of ...
1,0,Is the Solid Men Multicolor Track Pants by Yor...,Clothing and Accessories,York,Is the Solid Men Multicolor Track Pants by Yor...
2,1,What materials are used to make the Solid Men ...,Clothing and Accessories,York,What materials are used to make the Solid Men ...
3,1,Is the waistband of the Yorker track pants des...,Clothing and Accessories,York,Is the waistband of the Yorker track pants des...
4,2,What material is used in the York Solid Men Mu...,Clothing and Accessories,York,What material is used in the York Solid Men Mu...
5,2,Is the waistband of the York trackpants itch-f...,Clothing and Accessories,York,Is the waistband of the York trackpants itch-f...
6,3,What materials are used in the Yorker Solid Me...,Clothing and Accessories,York,What materials are used in the Yorker Solid Me...
7,3,Are the Yorker track pants comfortable for all...,Clothing and Accessories,York,Are the Yorker track pants comfortable for all...
8,4,What materials are used in the York Solid Men ...,Clothing and Accessories,York,What materials are used in the York Solid Men ...
9,4,Can you tell me about the sizing options avail...,Clothing and Accessories,York,Can you tell me about the sizing options avail...
